## Building AI assistants that scale using machine learning and open source tools

![alt text](https://encrypted-tbn0.gstatic.com/images?q=tbn:ANd9GcTaX3LNhGcAe1HnPZSuWS0oH6af0LJHXcH7If1sQgLCFAT1chNGFg)

In this workshop, you will learn how to build your own conversational AI assistant using machine learning and real conversational data. The goal of this workshop is to walk you through the process of building an ML-powered assistant from scratch and build an actual assistant which you can improve later.


There are no additional requirements to run this notebook, but if you encounter any issues or have more questions about the content included here, feel free send a message to an author of this exercise at juste@gmail.com

## Construindo assistentes de IA que escalam usando aprendizado de máquina e ferramentas de código aberto

! [texto alternativo] (https://encrypted-tbn0.gstatic.com/images?q=tbn:ANd9GcTaX3LNhGcAe1HnPZSuWS0oH6af0LJHXcH7If1sQgLCFAT1chNGFg)

Neste workshop, você aprenderá como construir seu próprio assistente de IA de conversação usando aprendizado de máquina e dados reais de conversação. O objetivo deste workshop é guiá-lo através do processo de construção de um assistente de ML a partir do zero e construir um assistente real que você possa melhorar mais tarde.


Não há requisitos adicionais para executar este notebook, mas se você encontrar quaisquer problemas ou tiver mais perguntas sobre o conteúdo incluído aqui, fique à vontade para enviar uma mensagem para um autor deste exercício em juste@gmail.com

### Introduction

### Introdução

During the course of this 3-hour workshop, you will go through each stage of the chatbot development and build an assistant capable of recommending talks to attend in this conference and answer the questions about the event. Below is an example conversation your assistant will be able to handle:

U: Hello  
A: Hello! I am a conference assistant ready to help you find talks to attend or anwers any questions you have.  
U: What session would you recommend for Data Scientists today?  
A: You can attend 'Building AI assistnats that scale using machine learning and open source tools' session!  
U: Who is the speaker and how long is the session?  
A: The speaker is Justina. The session is 3 hours long.  
U: Thanks!  
A: You are very welcome!  



      

The workshop consists of the following stages:

**0. Intro:**  
    0.1 Setup and installation 
      
**1. Stage 1: Natural language understanding:**  
    1.1. Designing the happy path  
    1.2. Generating the NLU training examples  
    1.3. Designing the training pipeline  
    1.4. Training the first NLU model  
    1.5.  Handling out-of-scope inputs  
    1.6. Adding sinonyms  
    1.7. Adding multi-intents  
    1.8. Re-training and thesting the updated NLU model  
      
**2. Stage 2: Dialogue management model:**  
    2.1. Designing training stories  
    2.2. Setting up the backend component  
    2.3. Creating a custom action  
    2.4. Defining the domain  
    2.5. Training the dialogue model  
    2.6. Testing the dialogue model   
    2.7. Handling out-of-scope conversations  
    2.8. Adding stories with multi-intents  
    2.9. Evaluating dialogue model  
      
**3. Stage 3: Closing the feedback loop:**  
    3.1. Improving the assistant using the interactive learning   
    3.2. Storing conversation history    
    3.3. Connecting the assistant to the outside world  
   

## 0. Intro
In this section, we will install all the necessary dependencies needed to successfully run this exercise.
### 0.1. Setup and installation
The best way to insall the necessary modules is to use the requirements.txt file. After creating a virtual environment, run:

**pip install -r requirements.txt**

Throughout this workshop, we will use only open source tools. The code block below checks if Rasa NLU and Rasa Core have been installed suffessfully.

## 0. Introdução
Nesta seção, instalaremos todas as dependências necessárias para executar este exercício com sucesso.
### 0.1. Instalação e instalação
A melhor maneira de instalar os módulos necessários é usar o arquivo requirements.txt. Depois de criar um ambiente virtual, execute:

**pip install -r requirements.txt**

Ao longo deste workshop, usaremos apenas ferramentas de código aberto. O bloco de código abaixo verifica se o Rasa NLU e o Rasa Core foram instalados sem problemas.

In [ ]:
import rasa_nlu
import rasa_core
import warnings
warnings.filterwarnings('ignore')


print("rasa_nlu: {} rasa_core: {}".format(rasa_nlu.__version__, rasa_core.__version__))

## 1. Natural Language Understanding 

## 1. Compreensão da Linguagem Natural

In this section, you will enable your assistant to understand the user inputs by building a Rasa NLU model. This model will take unstructured user inputs and extract structured data in a form of intents and entities:  
- *intent* - a label which represents the overall intention of the user 's input
- *entity* - important detail which an assistant should extract and use to steer the conversation

Nesta seção, você permitirá que seu assistente entenda as entradas do usuário construindo um modelo Rasa NLU. Esse modelo usará entradas não estruturadas do usuário e extrairá dados estruturados em uma forma de intenções e entidades:
- *intent* - um rótulo que representa a intenção geral da entrada do usuário
- *entidade* - detalhe importante que um assistente deve extrair e usar para orientar a conversa

### 1.1. Designing a happy path

### 1.1. Criando um caminho feliz

A good starting point is to define a happy path first. A happy path is a conversation flow where the user provides all the required information and allows the assistant to lead the conversation.

Um bom ponto de partida é definir um caminho feliz primeiro. Um caminho feliz é um fluxo de conversa em que o usuário fornece todas as informações necessárias e permite que o assistente conduza a conversa.

### 1.2. Designing the NLU training data

### 1.2. Projetando os dados de treinamento da NLU

To train the NLU model you will need some labeled training data. Rasa NLU training data samples consist of the following components:  
- intent label which starts with a prefix *
- examples of text inputs which correspond to that label
- entities which follow the format *[entity_value] (entity_label)*

We will start by generating some training data examples by hand. For a completed data file check out the *helper_files/nlu_data.md* in the repository of this exercise.

Para treinar o modelo NLU, você precisará de alguns dados de treinamento rotulados. As amostras de dados de treinamento Rasa NLU consistem dos seguintes componentes:
- etiqueta de intenção que começa com um prefixo *- exemplos de entradas de texto que correspondem a esse rótulo
- entidades que seguem o formato* [entidade _valor] (entidade_ rótulo) *Vamos começar gerando alguns exemplos de dados de treinamento manualmente. Para um arquivo de dados completo, confira o* helper _files / nlu_ data.md * no repositório deste exercício.

In [ ]:
nlu_md = """

## intent:greet
- hey
- hello there
- hi
- hello there
- good morning
- good evening
- moin
- hey there
- let's go
- hey dude
- goodmorning
- goodevening
- good afternoon

## intent:goodbye
- cu
- good by
- cee you later
- good night
- good afternoon
- bye
- goodbye
- have a nice day
- see you around
- bye bye
- see you later

## intent:recommend_session
- What presentation would you recommend to [data scientists](relevant_audience)?
- Which talks are relevant to people in [Machine Learning](relevant_audience) field?
- I work as a [product manager](relevant_audience). What sessions would you recommend for me to attend today?
- Are the any talks you could recommend to [machine learning](relevant_audience) folks to attend tomorrow?
- Which talks today are relevant to [developers](relevant_audience)?

## intent:speaker
- Who is the speaker?
- And who's presenting?
- What's the name of the presenter?
- Who's presenting?
- Who's speaking?
- The name of the speaker?

## intent:length
- How long is the session?
- And what's the length of this?
- How long is this session?
- Can you tell me how long this session is?
- Is the session long?

## intent:abstract
- Show me the abstract
- Can you give me more details about this talk?
- Is there a description of this presetnation?
- Can you show me an abstract of this talk?
- Show me the abstract, please
- Can you show me the summary of the talk?
- What this talk will be about?

## intent:thanks
- Thank you.
- very useful. thank you so much!
- Thanks
- thanks a lot
- thank you so much

## intent:inform
- to [Data Scientists](relevant_audience)
- relevant to [machine learning engineers](relevant_audience)
- for [product](relevant_audience) people
"""

%store nlu_md > nlu.md



## 1.3 Designing the training pipeline

Once the training data is ready, we can define the NLU model. We can do that by constructing the processing pipeline which defines how structured data will be extracted from unstructured user inputs: how the sentences will be tokenized, what intent classifier will be used, what entity extraction model will be used, etc. Each component in a training pipeline is trained one after another and can take inputs from the previously defined component as well as pass some information to subsequent ones.

## 1.3 Criando o pipeline de treinamento

Quando os dados de treinamento estiverem prontos, podemos definir o modelo NLU. Podemos fazer isso construindo o pipeline de processamento que define como os dados estruturados serão extraídos das entradas não estruturadas do usuário: como as sentenças serão tokenizadas, qual classificador de intenção será usado, qual modelo de extração de entidade será usado, etc. Cada componente em um O pipeline de treinamento é treinado um após o outro e pode receber entradas do componente previamente definido, assim como passar algumas informações para os subsequentes.

In [ ]:
configuration = """
language: "en"

pipeline:
- name: "tokenizer_whitespace"             # splits the sentence into tokens
- name: "ner_crf"                   # uses the pretrained spacy NER model
- name: "intent_featurizer_count_vectors"     # transform the sentence into a vector representation
- name: "intent_classifier_tensorflow_embedding"   # intent classifier
""" 

%store configuration > config.yml

## 1.4 Training the first Rasa NLU Model
Now, we're going to train the NLU model to recognise user inputs, so that when you send a message like "hello" to your bot, it will recognise this as a "greet" intent. Let's define the training function:

## 1.4 Treinando o primeiro modelo Rasa NLU
Agora, vamos treinar o modelo NLU para reconhecer as entradas do usuário, para que quando você enviar uma mensagem como "olá" para o seu bot, ele reconheça isso como uma intenção de "cumprimentar". Vamos definir a função de treinamento:

In [ ]:
from rasa_nlu.training_data import load_data
from rasa_nlu.config import RasaNLUModelConfig
from rasa_nlu.model import Trainer
from rasa_nlu import config

def train_nlu_model():
    # loading the nlu training samples
    training_data = load_data("nlu.md")

    # trainer to educate our pipeline
    trainer = Trainer(config.load("config.yml"))

    # train the model!
    interpreter = trainer.train(training_data)

    # store it for future use
    model_directory = trainer.persist("./models/current", fixed_model_name="nlu")
    
    return interpreter, model_directory



Finally, let's train the model using the previously defined data and model configuration:

Finalmente, vamos treinar o modelo usando dados previamente definidos e configuração do modelo:

In [ ]:
interpreter, model_directory = train_nlu_model()

## Testing the model

We have trained the first version of our NLU model! Let's test it on various inputs:

## Testando o modelo

Nós treinamos a primeira versão do nosso modelo NLU! Vamos testá-lo em várias entradas:

In [ ]:
import logging, io, json, warnings
logging.basicConfig(level="INFO")
warnings.filterwarnings('ignore')

def pprint(o):
    # small helper function to make dict dumps a bit prettier
    print(json.dumps(o, indent=2))

#change the input message with your prefered inputs
input_message = "What talks would you recommend to data scientists?"
pprint(interpreter.parse(input_message))

## Handling out-of-scope inputs
When dealing with conversational AI, out-of-scope user inputs are very common challenge. These inputs represent the user requests which have nothing to do with the assistant's job. While it's very challenging to provide a sensible answer to each out-of-scope input, it's important to enable your assistant to identify such inputs and guide the user back to the conversation. First, let's enable our assistant to identify out-of-scope inputs. To do that, we will add a new intent called *out-of-scope* to our training dataset and provde some corresponding inputs:

## Gerenciando entradas fora do escopo
Ao lidar com a IA conversacional, as entradas do usuário fora do escopo são um desafio muito comum. Essas entradas representam as solicitações do usuário que não têm nada a ver com o trabalho do assistente. Embora seja muito desafiador fornecer uma resposta sensata a cada entrada fora do escopo, é importante permitir que seu assistente identifique essas entradas e guie o usuário de volta à conversa. Primeiro, vamos permitir que nosso assistente identifique entradas fora do escopo. Para fazer isso, adicionaremos um novo intento chamado *fora do escopo* ao nosso conjunto de dados de treinamento e forneceremos algumas entradas correspondentes:

In [ ]:
nlu_md = """


## intent:greet
- hey
- hello there
- hi
- hello there
- good morning
- good evening
- moin
- hey there
- let's go
- hey dude
- goodmorning
- goodevening
- good afternoon

## intent:goodbye
- cu
- good by
- cee you later
- good night
- good afternoon
- bye
- goodbye
- have a nice day
- see you around
- bye bye
- see you later

## intent:recommend_session
- What presentation would you recommend to [data scientists](relevant_audience)?
- Which talks are relevant to people in [Machine Learning](relevant_audience) field?
- I work as a [product manager](relevant_audience). What sessions would you recommend for me to attend today?
- Are the any talks you could recommend to [machine learning](relevant_audience) folks to attend tomorrow?
- Which talks today are relevant to [developers](relevant_audience)?

## intent:speaker
- Who is the speaker?
- And who's presenting?
- What's the name of the presenter?
- Who's presenting?
- Who's speaking?
- The name of the speaker?

## intent:length
- How long is the session?
- And what's the length of this?
- How long is this session?
- Can you tell me how long this session is?
- Is the session long?

## intent:abstract
- Show me the abstract
- Can you give me more details about this talk?
- Is there a description of this presetnation?
- Can you show me an abstract of this talk?
- Show me the abstract, please
- Can you show me the summary of the talk?
- What this talk will be about?

## intent:thanks
- Thank you.
- very useful. thank you so much!
- Thanks
- thanks a lot
- thank you so much

## intent:inform
- to [Data Scientists](relevant_audience)
- relevant to [machine learning engineers](relevant_audience)
- for [product](relevant_audience) people

## intent:out-of-scope
- I want pizza
- please help with my ice cream it's dripping
- no wait go back i want a dripping ice cream but a cone that catches it so you can drink the ice cream later
- i want a non dripping ice cream
- hey little mama let em whisper in your ear
- someone call the police i think the bot died
- show me a picture of a chicken
- neither
- I want french cuisine
- i am hungry
- restaurants
- restaurant
- you're a loser lmao
- can i be shown a gluten free restaurant
- i don't care!!!!
- i do not care how are you
- again?
- oh wait i gave you my work email address can i change it?
- hang on let me find it
- stop it, i do not care!!!
- really? you're so touchy?
- how come?
- I changed my mind
- what?
- did i break you

"""

%store nlu_md > nlu.md


Let's retrain the model and see how it deals with out-of-scope inputs now:

Vamos treinar novamente o modelo e ver como ele lida com as entradas fora do escopo agora:

In [ ]:
interpreter, model_directory = train_nlu_model()

In [ ]:
input_message = "I want pizza"
pprint(interpreter.parse(input_message))

## 1.6 Adding synonyms

Synonyms are a very useful Rasa NLU feature which maps extracted entities to the same name. It's used when some extracted values have to be normalised so that they could be used to query the database or make an API call. In our example, the occupation of the relevant audience is a good candidate for the synonym because users can provide the same occupation in a variety of different ways (for example, Machine Learning and ML). Let's update our training examples with synonyms.

## 1.6 Adicionando sinônimos

Sinônimos são um recurso RUU muito útil que mapeia entidades extraídas com o mesmo nome. É usado quando alguns valores extraídos precisam ser normalizados para que possam ser usados ​​para consultar o banco de dados ou fazer uma chamada de API. Em nosso exemplo, a ocupação do público relevante é um bom candidato para o sinônimo, pois os usuários podem fornecer a mesma ocupação de várias maneiras diferentes (por exemplo, Machine Learning e ML). Vamos atualizar nossos exemplos de treinamento com sinônimos.

In [ ]:
nlu_md = """

## intent:greet
- hey
- hello there
- hi
- hello there
- good morning
- good evening
- moin
- hey there
- let's go
- hey dude
- goodmorning
- goodevening
- good afternoon

## intent:goodbye
- cu
- good by
- cee you later
- good night
- good afternoon
- bye
- goodbye
- have a nice day
- see you around
- bye bye
- see you later

## intent:recommend_session
- What presentation would you recommend to [data scientists](relevant_audience)?
- Which talks are relevant to people in [Machine Learning](relevant_audience:ML) field?
- I work as a [product manager](relevant_audience). What sessions would you recommend for me to attend today?
- Are the any talks you could recommend to [machine learning](relevant_audience:ML) folks to attend tomorrow?
- Which talks today are relevant to [developers](relevant_audience)?

## intent:speaker
- Who is the speaker?
- And who's presenting?
- What's the name of the presenter?
- Who's presenting?
- Who's speaking?
- The name of the speaker?

## intent:length
- How long is the session?
- And what's the length of this?
- How long is this session?
- Can you tell me how long this session is?
- Is the session long?

## intent:abstract
- Show me the abstract
- Can you give me more details about this talk?
- Is there a description of this presetnation?
- Can you show me an abstract of this talk?
- Show me the abstract, please
- Can you show me the summary of the talk?
- What this talk will be about?

## intent:thanks
- Thank you.
- very useful. thank you so much!
- Thanks
- thanks a lot
- thank you so much

## intent:inform
- to [Data Scientists](relevant_audience)
- relevant to [machine learning engineers](relevant_audience:ML)
- for [product](relevant_audience) people


## intent:out-of-scope
- I want pizza
- please help with my ice cream it's dripping
- no wait go back i want a dripping ice cream but a cone that catches it so you can drink the ice cream later
- i want a non dripping ice cream
- hey little mama let em whisper in your ear
- someone call the police i think the bot died
- show me a picture of a chicken
- neither
- I want french cuisine
- i am hungry
- restaurants
- restaurant
- you're a loser lmao
- can i be shown a gluten free restaurant
- i don't care!!!!
- i do not care how are you
- again?
- oh wait i gave you my work email address can i change it?
- hang on let me find it
- stop it, i do not care!!!
- really? you're so touchy?
- how come?
- I changed my mind
- what?
- did i break you
"""

%store nlu_md > nlu.md

To train the NLU model with synonyms, we have to add the synonyms component to the model pipeline:

Para treinar o modelo NLU com sinônimos, temos que adicionar o componente de sinônimos ao pipeline do modelo:

In [ ]:
configuration = """
language: "en"

pipeline:
- name: "tokenizer_whitespace"             # splits the sentence into tokens
- name: "ner_crf"                   # uses the pretrained spacy NER model
- name: "intent_featurizer_count_vectors"     # transform the sentence into a vector representation
- name: "intent_classifier_tensorflow_embedding"   # intent classifier
- name: "ner_synonyms"     #trains synonyms component
""" 

%store configuration > config.yml

Now, let's retrain the NLU model and test the performace.

Agora, vamos treinar novamente o modelo NLU e testar o desempenho.

In [ ]:
interpreter, model_directory = train_nlu_model()

See how 'machine learning engineers' now gets mapped to 'ML':

Veja como os 'engenheiros de aprendizado de máquina' agora são mapeados para 'ML':

In [ ]:
input_message = "For machine learning engineers"
pprint(interpreter.parse(input_message))

## 1.7 Implementing multi-intents

The NLU model we have built so far works pretty well, but it only supports inputs with only one intent per user input. In this step, we will use a tensorflow embedding model to enable the assistant to recognise multi-intents - more than one intention per user input. Let's start by defining multi-intents in our training data. Multi-intents are defined in a very similar way as regular intents, the only difference is that the label names consists of intent tokens and a character of your choice that separates them, for example **intent_token1+intent_token2**.

## 1.7 Implementando multi-intenções

O modelo NLU que construímos até agora funciona muito bem, mas suporta apenas entradas com apenas uma intenção por entrada de usuário. Nesta etapa, usaremos um modelo de incorporação de tensorflow para permitir que o assistente reconheça múltiplas intenções - mais de uma intenção por entrada de usuário. Vamos começar definindo as múltiplas intenções em nossos dados de treinamento. Multi-intenções são definidas de maneira muito semelhante às intenções regulares, a única diferença é que os nomes dos rótulos consistem em tokens de intenção e um caractere de sua escolha que os separa, por exemplo, **intent_token1 + intent_token2**.

In [ ]:
nlu_md = """

## intent:greet
- hey
- hello there
- hi
- hello there
- good morning
- good evening
- moin
- hey there
- let's go
- hey dude
- goodmorning
- goodevening
- good afternoon

## intent:goodbye
- cu
- good by
- cee you later
- good night
- good afternoon
- bye
- goodbye
- have a nice day
- see you around
- bye bye
- see you later

## intent:recommend_session
- What presentation would you recommend to [data scientists](relevant_audience)?
- Which talks are relevant to people in [Machine Learning](relevant_audience:ML) field?
- I work as a [product manager](relevant_audience). What sessions would you recommend for me to attend today?
- Are the any talks you could recommend to [machine learning](relevant_audience:ML) folks to attend tomorrow?
- Which talks today are relevant to [developers](relevant_audience)?

## intent:speaker
- Who is the speaker?
- And who's presenting?
- What's the name of the presenter?
- Who's presenting?
- Who's speaking?
- The name of the speaker?

## intent:length
- How long is the session?
- And what's the length of this?
- How long is this session?
- Can you tell me how long this session is?
- Is the session long?

## intent:abstract
- Show me the abstract
- Can you give me more details about this talk?
- Is there a description of this presetnation?
- Can you show me an abstract of this talk?
- Show me the abstract, please
- Can you show me the summary of the talk?
- What this talk will be about?

## intent:thanks
- Thank you.
- very useful. thank you so much!
- Thanks
- thanks a lot
- thank you so much

## intent:inform
- to [Data Scientists](relevant_audience)
- relevant to [machine learning engineers](relevant_audience:ML)
- for [product](relevant_audience) people


## intent:out-of-scope
- I want pizza
- please help with my ice cream it's dripping
- no wait go back i want a dripping ice cream but a cone that catches it so you can drink the ice cream later
- i want a non dripping ice cream
- hey little mama let em whisper in your ear
- someone call the police i think the bot died
- show me a picture of a chicken
- neither
- I want french cuisine
- i am hungry
- restaurants
- restaurant
- you're a loser lmao
- can i be shown a gluten free restaurant
- i don't care!!!!
- i do not care how are you
- again?
- oh wait i gave you my work email address can i change it?
- hang on let me find it
- stop it, i do not care!!!
- really? you're so touchy?
- how come?
- I changed my mind
- what?
- did i break you


## intent:speaker+length
 - Who is the presenter? Also, how long is the talk?
 - Who is the speaker and how long is the session?
 - Is the session long and who is presenting?
 - Do you know who is the presenter of the session? And how long is the session?
 - Is the talk long and who is presenting?
 - Who is the speaker? And how long is the talk?
"""

%store nlu_md > nlu.md


Next, let's modify the configuration of the model pipeline to use the tensorflow_embedding model with multi-intent support.

Em seguida, vamos modificar a configuração do pipeline de modelo para usar o modelo tensorflow_embedding com suporte a várias intenções.

In [ ]:
configuration = """
language: "en"

pipeline:
- name: "tokenizer_whitespace"             # splits the sentence into tokens
- name: "ner_crf"                   # uses the pretrained spacy NER model
- name: "intent_featurizer_count_vectors"     # transform the sentence into a vector representation
- name: "intent_classifier_tensorflow_embedding"   # intent classifier
  intent_tokenization_flag: true # sets multi-intent tokenization
  intent_split_symbol: "+"       # sets which symbol should be used for tokenization
- name: "ner_synonyms"   # trains synonyms component 
""" 

%store configuration > config.yml

Let's retrain the model with the new pipeline and test the performance:

Vamos treinar novamente o modelo com o novo pipeline e testar o desempenho:

In [ ]:
interpreter, model_directory = train_nlu_model()

See how a two-question input now gets recognised as a multi-intent:

Veja como uma entrada de duas perguntas agora é reconhecida como uma múltipla intenção:

In [ ]:
input_message = "Who is the speakers and how long is the session?"
pprint(interpreter.parse(input_message))

## 1.8 Evaluating the NLU model


Testing the model on various inputs is a good way to get high-level insights into the performance of the model. However, it's a time consuming and quite a tedious way of testing. Instead of evaluating the model by hand, it can also be evaluated on a test data set (though for simplicity we are going to use the same for test and train):

## 1.8 Avaliando o modelo NLU


Testar o modelo em várias entradas é uma boa maneira de obter insights de alto nível sobre o desempenho do modelo. No entanto, é uma maneira demorada e tediosa de testar. Em vez de avaliar o modelo manualmente, ele também pode ser avaliado em um conjunto de dados de teste (embora, por simplicidade, usemos o mesmo para teste e treinamento):

In [ ]:

from rasa_nlu.evaluate import run_evaluation
import IPython
from IPython import display

run_evaluation("nlu.md", model_directory)

Congratulations! You have just implemented the natural language understanding part of your assistant which means that your assistant can now understand you. In the second part of this workshop, we will delve into the next stage of the chatbot development - dialogue management.

Parabéns! Você acaba de implementar a parte de compreensão de linguagem natural de seu assistente, o que significa que seu assistente agora pode entender você. Na segunda parte deste workshop, vamos nos aprofundar na próxima etapa do desenvolvimento do chatbot - gerenciamento de diálogos.

# 2. Dialogue Management


# 2. Gerenciamento de Diálogo

In this section of this workshop you will build a machine learning-based dialogue model which will enable your assistant to decide on how to respond to user inputs based on the state of the conversation. 

Nesta seção deste workshop, você construirá um modelo de diálogo baseado em aprendizado de máquina que permitirá ao seu assistente decidir como responder às entradas do usuário com base no estado da conversa.

## 2.1 Designing the training stories

## 2.1 Criando as histórias de treinamento

Let's start with generating the training data. Rasa Core models learn by observing real conversational data between the user and the assistant. The only important thing is that this data has to be converted into the Rasa Core format: user inputs have to be expressed as corresponding intents (and entities where necessary) while the responses of the assistant are expressed as action names. Each training story follows the format:  
- the story starts with a story name which has a prefix ##  
- intents, corresponding to user inputs, start with *  
- if NLU model extracts entities which should influence the predictions of the dialogue model, they have to be included in the stories using the following format: * intent{'entity_name':"entity_value"}  
- the responses of the bot start with -  
- the story ends with an empty line which marks the end of the story

In the next step of this tutorial, we will generate some training stories to cover the happy path. To see a complete training data example, check out the **data/stories.md** file of this repository.


Vamos começar gerando os dados de treinamento. Os modelos Rasa Core aprendem observando dados reais de conversação entre o usuário e o assistente. A única coisa importante é que esses dados devem ser convertidos no formato Rasa Core: as entradas do usuário devem ser expressas como intenções correspondentes (e entidades, quando necessário), enquanto as respostas do assistente são expressas como nomes de ação. Cada história de treinamento segue o formato:
- a história começa com um nome de história que tem um prefixo ##
- intenções, correspondentes às entradas do usuário, começam com *- se o modelo NLU extrai entidades que devem influenciar as previsões do modelo de diálogo, elas devem ser incluídas nas histórias usando o seguinte formato:* intent {'entity _name': "entidade_ value"}
- as respostas do bot começam com -
- a história termina com uma linha vazia que marca o fim da história

Na próxima etapa deste tutorial, vamos gerar algumas histórias de treinamento para cobrir o caminho feliz. Para ver um exemplo completo de dados de treinamento, confira o arquivo **data / stories.md** deste repositório.

In [ ]:
stories_md = """
## happy path all info 1               
* greet              
  - utter_greet
* recommend_session{"relevant_audience":"Data Scientists"}               
  - action_recommend_session
  - slot{"speaker":"Justina"}
  - slot{"length":"5 min"}
  - slot{"abstract":"Workshop on chatbots"}
* length
  - utter_length
* thanks
  - utter_thanks
  
  
## happy path all info 2               
* greet              
  - utter_greet
* recommend_session{"relevant_audience":"Data Scientists"}               
  - action_recommend_session
  - slot{"speaker":"Justina"}
  - slot{"length":"5 min"}
  - slot{"abstract":"Workshop on chatbots"}
* length
  - utter_length
* speaker
  - utter_speaker
* thanks
  - utter_thanks    
  

## happy path all info 3               
* greet              
  - utter_greet
* recommend_session{"relevant_audience":"Data Scientists"}               
  - action_recommend_session
  - slot{"speaker":"Justina"}
  - slot{"length":"5 min"}
  - slot{"abstract":"Workshop on chatbots"}
* speaker
  - utter_speaker
* length
  - utter_length
* thanks
  - utter_thanks  


## happy path no relevant audience            
* greet              
  - utter_greet
* recommend_session
  - utter_ask_relevant_audience
* inform{"relevant_audience":"Data Scientists"}
  - action_recommend_session
  - slot{"speaker":"Justina"}
  - slot{"length":"5 min"}
  - slot{"abstract":"Workshop on chatbots"}
* length
  - utter_length
* speaker
  - utter_speaker
* thanks
  - utter_thanks 

"""

%store stories_md > stories.md

## 2.2 Setting up the backend component

We want to make our assistant engaging and fun. For that reason, we will enable it to answer the questions using the real data stored in a SQL database. For this exercise, the assistant will be able to pull information about the conference agenda, talks, and speakers. Let's take a look at how the data in a SQL database looks like.

## 2.2 Configurando o componente backend

Queremos tornar nossa assistente envolvente e divertida. Por esse motivo, permitiremos que ele responda às perguntas usando os dados reais armazenados em um banco de dados SQL. Para este exercício, o assistente poderá obter informações sobre a agenda, palestras e palestrantes da conferência. Vamos dar uma olhada em como os dados em um banco de dados SQL se parece.

In [ ]:
import sqlite3 as lite
import pandas as pd

conn = lite.connect('ConfDB.db')
data = pd.read_sql_query("SELECT * FROM agenda LIMIT 10", conn)
print(data)

## 2.3 Creating custom action

We are going to use the backend integration to enable our assistant to fetch the relevant data based on user's queries. For that, we will create custom actions which, when predicted, will collect necessary data and use it to steer the conversation further:

## 2.3 Criando uma ação personalizada

Vamos usar a integração de back-end para permitir que nosso assistente obtenha os dados relevantes com base nas consultas do usuário. Para isso, criaremos ações personalizadas que, quando previstas, coletarão os dados necessários e os usarão para direcionar a conversa ainda mais:

In [ ]:
actions = """
from rasa_core_sdk import Action
from rasa_core_sdk.events import SlotSet
import sqlite3 as lite
import pandas as pd
import time
from datetime import datetime
import random


class ActionRecommendTalk(Action):
    def name(self):
        return "action_recommend_session"
        
    def run(self, dispatcher, tracker, domain):
        conn = lite.connect('ConfDB.db')
        cur = conn.cursor()
        relevant_audience = tracker.get_slot('relevant_audience')
        cur.execute("SELECT * FROM agenda where relevant_audience like '%{}%'".format(relevant_audience))

        rows = cur.fetchall()
        ind = random.randint(0,len(rows))
        recommend_talk = list(rows[ind])
        title = recommend_talk[0]
        length = recommend_talk[8]
        speaker = recommend_talk[2]
        abstract = recommend_talk[4]
        
        dispatcher.utter_message("I would recommend you attend: {}".format(title))


        return [SlotSet("speaker", speaker), SlotSet("length",length), SlotSet("abstract", abstract)]

"""

%store actions > actions.py

## 2.4 Defining the domain

Once we have the training data in place, we can define the domain of our assistant. A domain defines the environment in which the assistant operates - what user inputs it should expect to see, what actions it should be able to predict, what information the assistant should store throughout the conversation.

## 2.4 Definindo o domínio

Assim que tivermos os dados de treinamento, podemos definir o domínio de nosso assistente. Um domínio define o ambiente no qual o assistente opera - quais entradas do usuário ele deve esperar ver, quais ações ele deve ser capaz de prever, quais informações o assistente deve armazenar durante toda a conversa.

In [ ]:
domain_yml = """
intents:
- greet
- goodbye
- recommend_session
- speaker
- length
- abstract
- speaker+length
- thanks
- out-of-scope
- inform



slots:
  relevant_audience:
    type: text
  venue:
    type: unfeaturized
  length:
    type: unfeaturized
  speaker:
    type: unfeaturized
  abstract:
    type: unfeaturized
    
entities:
- relevant_audience


actions:
- utter_greet
- utter_thanks
- utter_goodbye
- utter_length
- utter_abstract
- utter_speaker
- utter_ask_relevant_audience
- utter_out_of_scope
- utter_ask_other_questions
- action_recommend_session


templates:
  utter_greet:
    - text: "Hey! I am a conference assistant. I can help you find the sessions to attend, or answer conference-related questions."

  utter_thanks:
    - text: "You are very welcome!"

  utter_goodbye:
    - text: "See you later"
  
  utter_out_of_scope:
    - text: "Sorry, I can't help you with that"
  
  utter_ask_other_questions:
    - text: "Would you like to know anything else?"
    
  utter_ask_relevant_audience:
    - text: "What would be the relevant audience?"
  
  utter_speaker:
    - text: "The speaker is {speaker}."
  
  utter_length:
    - text: "The session is {length}"
    
  utter_abstract:
    - text: "{abstract}"

"""

%store domain_yml > domain.yml

## 2.5 Training the dialogue model

We now have all the components necessary to train the dialogue management model. The code cell below will train the model using the defined policy and store the model in a specified location for us to test later.

## 2.5 Treinando o modelo de diálogo

Agora temos todos os componentes necessários para treinar o modelo de gerenciamento de diálogos. A célula de código abaixo irá treinar o modelo usando a política definida e armazenar o modelo em um local especificado para que possamos testar mais tarde.

In [ ]:
from rasa_core.policies import KerasPolicy, MemoizationPolicy
from rasa_core.agent import Agent

def train_dialogue():
    # loading our neatly defined training dialogues
    agent = Agent("domain.yml", policies=[MemoizationPolicy(), KerasPolicy(epochs=200, max_history = 6)])
    training_data = agent.load_data('stories.md')


    agent.train(
        training_data)

    agent.persist('models/dialogue')


In [ ]:
train_dialogue()

## 2.6 Testing the dialogue model

It's finally time for the most exciting part - testing the bot! Let's spin up the custom action server and we are ready to go. Open a new terminal and exacute the following command:

**python -m rasa_core_sdk.endpoint --actions actions**

## 2.6 Testando o modelo de diálogo

Finalmente chegou a hora da parte mais emocionante - testar o bot! Vamos acelerar o servidor de ações personalizadas e estamos prontos para começar. Abra um novo terminal e exija o seguinte comando:

**python -m rasa_core_sdk.endpoint --actions actions**

In [ ]:
import IPython
from IPython.display import clear_output
from rasa_core.agent import Agent
from rasa_core.interpreter import NaturalLanguageInterpreter
from rasa_core.utils import EndpointConfig
import time

def load_assistant():
    messages = ["Hi! you can chat in this window. Type 'stop' to end the conversation."]
    interpreter = NaturalLanguageInterpreter.create(model_directory)
    endpoint = EndpointConfig('http://localhost:5055/webhook')
    agent = Agent.load('models/dialogue', interpreter=interpreter, action_endpoint = endpoint)

    print("Your bot is ready to talk! Type your messages here or send 'stop'")
    while True:
        a = input()
        if a == 'stop':
            break
        responses = agent.handle_text(a)
        for response in responses:
            print(response["text"])

In [ ]:
load_assistant()

## 2.7 Adding stories with multi-intents

Next, let's add a few stories with multi-intents. Such stories will follow a regular data format, the only thing is that we can include a couple of actions to be predicted by an assistant:

## 2.7 Adicionando histórias com várias intenções

Em seguida, vamos adicionar algumas histórias com várias intenções. Tais histórias seguirão um formato de dados regular, a única coisa é que podemos incluir algumas ações a serem previstas por um assistente:

## 2.7 Adicionando histórias com várias intenções

Em seguida, lettuce some stories with several intenções. Tais datas em um formato de dados regular, a única coisa que é possível incluir várias ações que são previstas por um assistente:

In [ ]:
stories_md = """
               
## happy path all info 1               
* greet              
  - utter_greet
* recommend_session{"relevant_audience":"Data Scientists"}               
  - action_recommend_session
  - slot{"speaker":"Justina"}
  - slot{"length":"5 min"}
  - slot{"abstract":"Workshop on chatbots"}
* length
  - utter_length
* thanks
  - utter_thanks
  
  
## happy path all info 2               
* greet              
  - utter_greet
* recommend_session{"relevant_audience":"Data Scientists"}               
  - action_recommend_session
  - slot{"speaker":"Justina"}
  - slot{"length":"5 min"}
  - slot{"abstract":"Workshop on chatbots"}
* length
  - utter_length
* speaker
  - utter_speaker
* thanks
  - utter_thanks    
  

## happy path all info 3               
* greet              
  - utter_greet
* recommend_session{"relevant_audience":"Data Scientists"}               
  - action_recommend_session
  - slot{"speaker":"Justina"}
  - slot{"length":"5 min"}
  - slot{"abstract":"Workshop on chatbots"}
* speaker
  - utter_speaker
* length
  - utter_length
* thanks
  - utter_thanks  


## happy path no relevant audience            
* greet              
  - utter_greet
* recommend_session
  - utter_ask_relevant_audience
* inform{"relevant_audience":"Data Scientists"}
  - action_recommend_session
  - slot{"speaker":"Justina"}
  - slot{"length":"5 min"}
  - slot{"abstract":"Workshop on chatbots"}
* length
  - utter_length
* speaker
  - utter_speaker
* thanks
  - utter_thanks 
  
## multi-intents story1              
* greet              
  - utter_greet
* recommend_session{"relevant_audience":"Data Scientists"}               
  - action_recommend_session
  - slot{"speaker":"Justina"}
  - slot{"length":"5 min"}
  - slot{"abstract":"Workshop on chatbots"}
* speaker+length
  - utter_length
  - utter_speaker
* thanks
  - utter_thanks
  
## multi-intents story2              
* greet              
  - utter_greet
* recommend_session{"relevant_audience":"Data Scientists"}               
  - action_recommend_session
  - slot{"speaker":"Justina"}
  - slot{"length":"5 min"}
  - slot{"abstract":"Workshop on chatbots"}
* speaker+length
  - utter_length
  - utter_speaker
* abstract
  - utter_abstract
* thanks
  - utter_thanks
  


"""

%store stories_md > stories.md

In [ ]:
train_dialogue()

In [ ]:
load_assistant()

## 2.8 Adding out-of-scope inputs

Finally, let's design a story with out-of-scope user inputs. Here, it's important to enable an assistant to take charge of the conversation and guide the user back to the initial conversation. In our case, an assistant will let the user know that it cannot deal with the out-of-scope request and will offer other questions to be asked:

In [ ]:
stories_md = """
## happy path all info 1               
* greet              
  - utter_greet
* recommend_session{"relevant_audience":"Data Scientists"}               
  - action_recommend_session
  - slot{"speaker":"Justina"}
  - slot{"length":"5 min"}
  - slot{"abstract":"Workshop on chatbots"}
* length
  - utter_length
* thanks
  - utter_thanks
  
  
## happy path all info 2               
* greet              
  - utter_greet
* recommend_session{"relevant_audience":"Data Scientists"}               
  - action_recommend_session
  - slot{"speaker":"Justina"}
  - slot{"length":"5 min"}
  - slot{"abstract":"Workshop on chatbots"}
* length
  - utter_length
* speaker
  - utter_speaker
* thanks
  - utter_thanks    
  

## happy path all info 3               
* greet              
  - utter_greet
* recommend_session{"relevant_audience":"Data Scientists"}               
  - action_recommend_session
  - slot{"speaker":"Justina"}
  - slot{"length":"5 min"}
  - slot{"abstract":"Workshop on chatbots"}
* speaker
  - utter_speaker
* length
  - utter_length
* thanks
  - utter_thanks  


## happy path no relevant audience            
* greet              
  - utter_greet
* recommend_session
  - utter_ask_relevant_audience
* inform{"relevant_audience":"Data Scientists"}
  - action_recommend_session
  - slot{"speaker":"Justina"}
  - slot{"length":"5 min"}
  - slot{"abstract":"Workshop on chatbots"}
* length
  - utter_length
* speaker
  - utter_speaker
* thanks
  - utter_thanks 

  
## multi-intents story1              
* greet              
  - utter_greet
* recommend_session{"relevant_audience":"Data Scientists"}               
  - action_recommend_session
  - slot{"speaker":"Justina"}
  - slot{"length":"5 min"}
  - slot{"abstract":"Workshop on chatbots"}
* speaker+length
  - utter_length
  - utter_speaker
* thanks
  - utter_thanks
  
## multi-intents story2              
* greet              
  - utter_greet
* recommend_session{"relevant_audience":"Data Scientists"}               
  - action_recommend_session
  - slot{"speaker":"Justina"}
  - slot{"length":"5 min"}
  - slot{"abstract":"Workshop on chatbots"}
* speaker+length
  - utter_length
  - utter_speaker
* abstract
  - utter_abstract
* thanks
  - utter_thanks
  
## multi-intents story2              
* greet              
  - utter_greet
* recommend_session{"relevant_audience":"Data Scientists"}               
  - action_recommend_session
  - slot{"speaker":"Justina"}
  - slot{"length":"5 min"}
  - slot{"abstract":"Workshop on chatbots"}
* speaker+length
  - utter_length
  - utter_speaker
* abstract
  - utter_abstract
* thanks
  - utter_thanks
  
  
## multi-intents story2              
* greet              
  - utter_greet
* recommend_session{"relevant_audience":"Data Scientists"}               
  - action_recommend_session
  - slot{"speaker":"Justina"}
  - slot{"length":"5 min"}
  - slot{"abstract":"Workshop on chatbots"}
* speaker+length
  - utter_length
  - utter_speaker
* abstract
  - utter_abstract
* thanks
  - utter_thanks
  
  
  
## out-of-scope input story 1            
* greet              
  - utter_greet
* recommend_session{"relevant_audience":"Data Scientists"}               
  - action_recommend_session
  - slot{"speaker":"Justina"}
  - slot{"length":"5 min"}
  - slot{"abstract":"Workshop on chatbots"}
* out-of-scope
  - utter_out_of_scope
  - utter_ask_other_questions
* abstract
  - utter_abstract
* thanks
  - utter_thanks
  
## out-of-scope input story 2            
* greet              
  - utter_greet
* recommend_session{"relevant_audience":"Data Scientists"}               
  - action_recommend_session
  - slot{"speaker":"Justina"}
  - slot{"length":"5 min"}
  - slot{"abstract":"Workshop on chatbots"}
* out-of-scope
  - utter_out_of_scope
  - utter_ask_other_questions
* thanks
  - utter_thanks 
  
## out-of-scope input story              
* greet              
  - utter_greet
* recommend_session{"relevant_audience":"Data Scientists"}               
  - action_recommend_session
  - slot{"speaker":"Justina"}
  - slot{"length":"5 min"}
  - slot{"abstract":"Workshop on chatbots"}
* out-of-scope
  - utter_out_of_scope
  - utter_ask_other_questions



"""

%store stories_md > stories.md

In [ ]:
train_dialogue()

In [ ]:
load_assistant()

## 2.9 Dialogue model evaluation

## 2.9 Avaliação do modelo de diálogo

Another great way to see how good our dialogue model is, is to test it using evaluation scripts:

Outra ótima maneira de ver como nosso modelo de diálogo é bom é testá-lo usando scripts de avaliação:

In [ ]:

!python -m rasa_core.evaluate --core models/dialogue --stories stories.md -o results


# 3. Closing the feedback loop

# 3. Fechando o ciclo de feedback

Developing an assistant is just one part of the process. Another very important part which defines a successful assistant is enabling your assistant to learn from real user feedback. In the last part of this workshop, we will cover two ways to improve your bots using real user feedback - using interactive learning and using the history of the conversations. We will also, connect our assistant to a custom webpage to see how it works in action! We will complete this part using the command line.

Desenvolver um assistente é apenas uma parte do processo. Outra parte muito importante que define um assistente de sucesso é permitir que seu assistente aprenda com o feedback real do usuário. Na última parte deste workshop, abordaremos duas maneiras de melhorar seus bots usando o feedback real do usuário - usando o aprendizado interativo e usando o histórico das conversas. Nós também conectaremos nosso assistente a uma página da Web personalizada para ver como ela funciona em ação! Nós completaremos esta parte usando a linha de comando.

## 3.1. Improving the assistant using the interactive learning 
Interactive learning is a great way to improve your assistant and generate more training example by simply talking to your bot and providing feedback for all predictions it made. That is the main idea behind it - instead of responding right away, an assistant will tell you what it thinks it should do next and ask you for feedback. To start the interactive learning session, we will use a command line and use the following command:


**python -m rasa_core.train interactive --core models/dialogue --nlu models/current/default/nlu --endpoints endpoints.yml**

## 3.1. Melhorando o assistente usando o aprendizado interativo
O aprendizado interativo é uma ótima maneira de melhorar seu assistente e gerar mais exemplos de treinamento simplesmente conversando com seu bot e fornecendo feedback para todas as previsões que ele fez. Essa é a idéia principal por trás disso - em vez de responder imediatamente, um assistente lhe dirá o que acha que deve fazer em seguida e pedir um feedback. Para iniciar a sessão de aprendizado interativo, usaremos uma linha de comando e usaremos o seguinte comando:


**python -m rasa_core.train interactive --modelos de core / dialog --nlu models / current / default / nll --endpoints endpoints.yml**

## 3.2. Storing conversation history 
Another way to improve your assistant is to observe real conversations between the users and a bot. To do so, we have to store the conversations in a storage first. In Rasa Core, tracker is responsible of keeping track of everything that happends throughout the conversation - user inputs, NLU model results, dialogue model predictions, etc. We can easily store all this data in a database for later use. In this step you will learn how to store this information in a Mongo tracker store. We will complete this step in a command line.

First, we will setup a mongodb backend and store the conversaton history there. For that, we will start our assistant on a server using:  
**python -m rasa_core.run -d models/dialogue -u models/current/default/nlu --port 5005  --endpoints endpoints.yml**

## 3.2. Armazenando histórico de conversas
Outra maneira de melhorar seu assistente é observar conversas reais entre os usuários e um bot. Para fazer isso, temos que armazenar as conversas em um armazenamento primeiro. No Rasa Core, o rastreador é responsável por acompanhar tudo o que acontece durante a conversa - entradas do usuário, resultados do modelo NLU, previsões de modelos de diálogo, etc. Podemos armazenar facilmente todos esses dados em um banco de dados para uso posterior. Nesta etapa, você aprenderá como armazenar essas informações em uma loja de rastreador Mongo. Nós completaremos este passo em uma linha de comando.

Primeiro, vamos configurar um backend mongodb e armazenar o histórico de conversação lá. Para isso, vamos iniciar nosso assistente em um servidor usando:
**python -m rasa_core.run -d models / diálogo -u models / current / default / nll --port 5005 --endpoints endpoints.yml**

##  3.3. Connecting the assistant to the outside world
In the very last step of this workshop, you will learn how to connect your assistant to a custom UI which can be easily added to a website of your choice. The repository of this workshop contains a folder called *bot_ui* where you can find a very basic html webpage. We will add a UI code stored in a *ui.html* file and connect our assistant to it. We will complete this step using a text editor and a command line.

After setting up the backed, we will start our assistant on a server and connect to the UI using:

**python -m rasa_core.run -d models/dialogue -u models/current/default/nlu --port 5005  --endpoints endpoints.yml --credentials credentials.yml**

## 3.3. Conectando o assistente ao mundo exterior
Na última etapa deste workshop, você aprenderá a conectar seu assistente a uma interface de usuário personalizada, que pode ser facilmente adicionada a um site de sua escolha. O repositório deste workshop contém uma pasta chamada *bot_ui* onde você pode encontrar uma página html bem básica. Vamos adicionar um código da interface do usuário armazenado em um arquivo *ui.html* e conectar nosso assistente a ele. Vamos concluir esta etapa usando um editor de texto e uma linha de comando.

Depois de configurar o backup, iniciaremos nosso assistente em um servidor e nos conectaremos à interface do usuário usando:

**python -m rasa_core.run -d models / diálogo -u models / current / default / nll --port 5005 --endpoints endpoints.yml --credentials credentials.yml**

# 4. Summary 
In this workshop, we covered some of the most important steps of the chatbot development and you should have a simple conference bot running your machine. There are so many things you can do to take this assistant to the whole new level! Here are some ideas for you:
- Add new skills like:
        show the session timetable 
        tell what is the next session at a specific venue   
        connect with a speaker on social media  
        recommend resources to learn more about the topic  
        
- Add new entities like date and time
- Connect your assistant to the most popular messaging platforms like Facebook, Slack or Telegram

Make sure to reference [Rasa official documentation](https://rasa.com/docs) or ask questions on the [Rasa Community Forum](https://forum.rasa.com) if you are in doubt! 

Most importantly, let me know what you came up with!

# 4. Resumo
Neste workshop, cobrimos algumas das etapas mais importantes do desenvolvimento do chatbot e você deve ter um simples bot de conferência executando sua máquina. Há tantas coisas que você pode fazer para levar este assistente a todo o novo nível! Aqui estão algumas idéias para você:
- Adicione novas habilidades como:
        mostre o calendário da sessão
        diga qual é a próxima sessão em um local específico
        conectar-se com um alto-falante nas mídias sociais
        recomendar recursos para aprender mais sobre o tópico
        
- Adicionar novas entidades, como data e hora
- Conecte seu assistente às plataformas de mensagens mais populares, como Facebook, Slack ou Telegram

Certifique-se de referenciar [documentação oficial do Rasa] (https://rasa.com/docs) ou fazer perguntas no [Fórum da Comunidade Rasa] (https://forum.rasa.com) se você estiver em dúvida!

Mais importante, deixe-me saber o que você veio com!